In [175]:
# 필요 라이브러리 호출
import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# 실패작
searchKeyWord = ""
searchWindowUrl = "https://www.data.go.kr/search/index.do?index=OPENAPI&query=&currentPage=1&countPerPage=10"
searchWindowRequests = requests.get(searchWindowUrl)
searchWindowText = searchWindowRequests.text

searchWindowBeautify = BeautifulSoup(searchWindowText, 'lxml')

searchWindowBeautify.select('#resultQuery')

BeautifulSoup(requests.post(searchWindowUrl).text, 'lxml').select('#resultQuery')

# URL 수집 자동화 프로젝트

In [ ]:
# 데이터 목록 불러오기
dataList = pd.read_csv("https://docs.google.com/spreadsheets/d/1VngqG-m7G8k1587c21MZoheR1Fz3amp1mJtiBvA1Jb0/export?format=csv&gid=0")

In [223]:
# 원하는 행 조회(공공데이터포털, 한국감정원)
publicDataPortal = dataList[(dataListConnectSite == '공공데이터포털') | (dataListConnectSite == '한국감정원')]

# 원하는 컬럼 조회
dataNameList = publicDataPortal['제공항목(데이터셋)'].tolist()
originalSourceList = publicDataPortal['원본제공처'].tolist()

In [216]:
# https://beomi.github.io/gb-crawling/posts/2017-02-27-HowToMakeWebCrawler-With-Selenium.html
# 위 링크에 따라 크롬 드라이버 불러오기
# (사용하고 있는 크롬의 버전에 맞는 드라이버를 다운받아서 경로 지정)
driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')

In [218]:
# 로그인 주소에 접속
driver.get('https://www.data.go.kr/subMain.jsp#/L2xvZ2luL2xvZ2luT3BlbiRAXjE3MW03MQ==')

In [219]:
# 로그인 아이디 및 비밀번호 입력
myId = 'nano8990'
myPassword = 'ehdwnsdl1!'

# 각 로그인 창에 입력된 아이디와 비밀번호 전송
driver.find_element_by_id('ID_x').send_keys(myId)
driver.find_element_by_id('PassWord_x').send_keys(myPassword)

# 로그인 버튼 클릭
driver.find_element_by_class_name('log').click()

In [220]:
# 각종 변수 설정
searchWindowUrl = 'https://www.data.go.kr/search/index.do'
searchBoxId = 'resultQuery'
searchButtonXpath = '//*[@onclick="javascript:resultQuerySearch()"]'

fileDataFormatDivisionTag = "div #file-list-wrapper .data-item"
openApiFormatDivisionTag = 'div #openapiDivArea > div .data-item'

dataNameTag = '<a href=".*">.*</a>'
originalSourceSpanTag = '<span class="category-filter visible-desktop">.*</span>'

In [221]:
# 검색창 접속
driver.get(searchWindowUrl)

In [224]:
# 검색하고자 하는 정보(데이터명, 원본제공자) 입력
eachDataName = dataNameList[1]
eachOriginalSource = originalSourceList[1]

# 검색창에 자료명 입력
searchBox = driver.find_element_by_id(searchBoxId)
searchBox.send_keys(eachDataName)

In [225]:
# 검색버튼 클릭
driver.find_element_by_xpath(searchButtonXpath).click()

In [ ]:
# 검색 완료창의 정보 획득
pageSource = driver.page_source
beautifulPageSource = BeautifulSoup(pageSource, 'html.parser')

## 파일 데이터 검색결과 처리

In [280]:
# 검색 결과(파일 데이터)에서 자료명 위치 검색
fileDataList = beautifulPageSource.select(fileDataFormatDivisionTag)
searchList = []

# 검색하고자 하는 정보를 담고 있는 태그를 컴파일
compileKeyDataNameTag = re.compile(dataNameTag, re.DOTALL)
compileKeyOriginalSource = re.compile(originalSourceSpanTag, re.DOTALL)

# 검색하고자 하는 정보를 컴파일
compileKeyDataName = re.compile(eachDataName)
compileKeyOriginalSource = re.compile(eachOriginalSource)

# 반복문을 돌린 후 파일데이터 존재여부 확인을 위해 None 설정
searchIndex = None

In [285]:
# 검색된 파일 데이터 수 만큼 반복
for j in range(0, len(fileDataList)):
    # 파일 데이터의 페이지 소스를 String 타입으로 변환
    fileDataStringType = str(fileDataList[j])
    # 페이지 소스에서 데이터명을 담고 있는 태그 검색
    dataNameTagSearchResult = compileKeyDataNameTag.search(fileDataStringType).group()
    # 태그에서 검색하고자 하는 자료명 검색
    if None != compileKeyDataName.search(dataNameTagSearchResult):
        # 페이지 소스에서 원본제공자를 담고 있는 태그 검색
        originalSourceSpanTagSearchResult = compileKeyOriginalSource.search(fileDataStringType).group()
        # 태그에서 검색하고자 하는 원본제공자 검색
        if None != compileKeyOriginalSource.search(originalSourceSpanTagSearchResult):
            searchIndex = j
            break

## 오픈 API 검색결과 처리

In [163]:
soup.select(openApiFormatDivisionTag)

[<div class="data-item">
 <span class="category-filter visible-desktop">
                                                     
                                                         
                                                         
                                                             국토관리
                                                         
                                                     
                                                 </span>
 <div class="data-title">
 <a href="/dataset/15002266/openapi.do">
                                                             한국감정원 부동산 매매가격지수 현황
                                                     </a>
 <span>조회수 : 666</span>
 <span>활용신청건수 : 98</span>
 <span class="category-filter visible-mobile">
                                                         
                                                             
                                                             
                                     